In [1]:
import requests;

In [9]:
with open(r"C:\Users\Saksham\Desktop\CC_task\pokemon.txt","r") as pk:
    content = pk.readlines()
    print(content)

['Amoonguss\n', 'Gyarados\n', '143\n', 'Volcarona\n', 'Wooper\n', 'Greninja\n', '231\n', 'Charizard\n', 'Sceptile\n', '681\n', 'Blaziken\n', 'Mudkip\n', 'Rowlet\n', '292\n', 'Wynaut\n', 'Incineroar\n', 'Spheal\n', 'Buzzwole\n', 'Salamence\n', '778\n', 'Metagross\n', 'Rayquaza\n', 'Falinks\n', 'Infernape\n', 'Piplup\n', 'Dragapult\n', 'Darkrai\n', 'Gholdengo\n', '501\n', 'Walking-Wake\n', 'Milotic\n', 'Bulbasaur\n', 'Gengar\n', 'Umbreon\n', 'Garchomp\n', 'Chandelure\n', 'Lucario\n', 'Flygon\n', 'Scizor\n', 'Squirtle\n', 'Dragonite\n', 'Luxray\n', 'Lugia\n', 'Noivern\n', 'Ceruledge\n', 'Haxorus\n', 'Psyduck\n', 'Shedinja\n', 'Annihilape\n', 'Magikarp']


In [51]:
len(content)

50

In [46]:
my_dict={}

In [48]:
for i in range(0,50): #50 is the length
    name_id = content[i].strip()
    r1=requests.get(f"https://pokeapi.co/api/v2/pokemon/{name_id}/")
    name=r1.json()["name"]
    idno=r1.json()["id"]
    type_=[]
    abilities=[]
    for ability in r1.json()["abilities"]:
            abilities.append(ability['ability']['name'])
    for types in r1.json()["types"]:
            type_.append(types['type']['name'])
    r2=requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{name_id}/")
    legendary=r2.json()["is_legendary"]
    mythical=r2.json()["is_mythical"]
    temp_dict={"id":idno,"is_legendary":legendary,"is_mythical":mythical,"abilities":abilities,"types":type_}
    my_dict[name]=temp_dict

In [91]:
damage_array={
    "attackers": [],
    "defenders": []
}
for j in range(1,19):
    r3=requests.get(f"https://pokeapi.co/api/v2/type/{j}/")
    name=r3.json()['name']
    for k in (r3.json()['damage_relations']):
            temp_attacker={}
            temp_defender={}
            for ele1 in r3.json()['damage_relations']["double_damage_from"]:
                        temp_defender[ele1["name"]]=2
            for ele1 in r3.json()['damage_relations']["double_damage_to"]:
                        temp_attacker[ele1["name"]]=2
            for ele1 in r3.json()['damage_relations']["half_damage_from"]:
                        temp_defender[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["half_damage_to"]:
                        temp_attacker[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["no_damage_from"]:
                        temp_defender[ele1["name"]]=0
            for ele1 in r3.json()['damage_relations']["no_damage_to"]:
                        temp_attacker[ele1["name"]]=0
    if (temp_attacker):
            damage_array["attackers"].append({name:temp_attacker})
            #print(temp_attacker)
    if (temp_defender):
            damage_array["defenders"].append({name:temp_defender})
         #print(temp_defender)
damage_array


{'attackers': [{'normal': {'rock': 0.5, 'steel': 0.5, 'ghost': 0}},
  {'fighting': {'normal': 2,
    'rock': 2,
    'steel': 2,
    'ice': 2,
    'dark': 2,
    'flying': 0.5,
    'poison': 0.5,
    'bug': 0.5,
    'psychic': 0.5,
    'fairy': 0.5,
    'ghost': 0}},
  {'flying': {'fighting': 2,
    'bug': 2,
    'grass': 2,
    'rock': 0.5,
    'steel': 0.5,
    'electric': 0.5}},
  {'poison': {'grass': 2,
    'fairy': 2,
    'poison': 0.5,
    'ground': 0.5,
    'rock': 0.5,
    'ghost': 0.5,
    'steel': 0}},
  {'ground': {'poison': 2,
    'rock': 2,
    'steel': 2,
    'fire': 2,
    'electric': 2,
    'bug': 0.5,
    'grass': 0.5,
    'flying': 0}},
  {'rock': {'flying': 2,
    'bug': 2,
    'fire': 2,
    'ice': 2,
    'fighting': 0.5,
    'ground': 0.5,
    'steel': 0.5}},
  {'bug': {'grass': 2,
    'psychic': 2,
    'dark': 2,
    'fighting': 0.5,
    'flying': 0.5,
    'poison': 0.5,
    'ghost': 0.5,
    'steel': 0.5,
    'fire': 0.5,
    'fairy': 0.5}},
  {'ghost': {'ghost': 

{'id': 491, 'is_legendary': False, 'is_mythical': True, 'abilities': ['bad-dreams'], 'types': ['dark']}
